In [ ]:
import os
import sys

from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import SimpleITK as sitk
import nrrd
import vtk

import torch
from torch import nn
from torch.nn import functional as F
from torchvision import transforms

import pytorch_lightning as pl
import pickle
import monai 
import glob 
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

sys.path.append('/mnt/famli_netapp_shared/C1_ML_Analysis/src/famli-ultra-sim/')
sys.path.append('/mnt/famli_netapp_shared/C1_ML_Analysis/src/famli-ultra-sim/dl')
import dl.transforms.ultrasound_transforms as ultrasound_transforms
import dl.loaders.mr_us_dataset as mr_us_dataset
import dl.nets.us_simulation_jit as us_simulation_jit
import dl.nets.us_simu as us_simu

import importlib

from dl.nets.layers import TimeDistributed

from torchvision import transforms as T


In [ ]:
mount_point = '/mnt/raid/C1_ML_Analysis'

In [ ]:
df = pd.read_csv('/mnt/famli_netapp_shared/C1_ML_Analysis/src/diffusion-models/blender/studies/placenta/ALL_C1_cines_gt_ga_withmeta_full_tags_sample_masked_resampled_256_spc075.csv')
df_study = df.query('study_id == "FAM-025-0754-2"')

In [ ]:
# model_fn = os.path.join(mount_point, 'train_output/simu_reconstruction/USPCReconstruction/v0.2/epoch=7-val_loss=0.01.ckpt')
# model_fn = os.path.join(mount_point, 'train_output/simu_reconstruction/USPCReconstruction/v0.3/epoch=8-val_loss=0.01.ckpt')
model_fn = os.path.join(mount_point, 'train_output/simu_reconstruction/USPCReconstruction/v0.6/last.ckpt')
pc_r = us_simu.USPCReconstruction.load_from_checkpoint(model_fn).eval().cuda()


In [ ]:
def sweep_resample(sweep_t, target_shape=(200, 256, 256)):
    return F.interpolate(sweep_t, target_shape, mode='nearest')   

sweeps_fov = []
sweeps_t = []
df_study_tags = df_study[df_study['tag'].isin(pc_r.vs.tags)]


inverse_transform_us = T.Compose([T.Pad((0, 0, 0, 40)),  T.Lambda(lambda x: T.functional.crop(x, 40, 0, 256, 256))])

for idx, row in df_study_tags.iterrows():
    tag = row['tag']
    sweep_fn = os.path.join(mount_point, row['file_path'])
    img_np, _ = nrrd.read(sweep_fn, index_order='C')
    img_np = img_np.astype(np.float32)
    sweep_t = torch.tensor(img_np).cuda().float()/255.0
    sweep_t = sweep_t[:, :, :, 0:1].permute(3, 0, 1, 2)
    
    sweep_t = inverse_transform_us(sweep_t.permute(1, 0, 2, 3)).permute(1, 0, 2, 3)
    
    sweep_resampled = sweep_resample(sweep_t.unsqueeze(0))
    # sweeps_fov.append(sweep_resampled)
    
    sweeps_fov.append(pc_r.vs.simulated_sweep_in_fov(tag, sweep_resampled))

# sweeps_fov = torch.stack(sweeps_fov)
# print(sweeps_fov[0].shape)
# print(sweeps_fov[0].shape)
# fig = px.imshow(sweeps_t[0].squeeze().cpu().numpy(), animation_frame=1, binary_string=True)
# fig.show()

# fig = px.imshow(sweeps_fov[0].squeeze().cpu().numpy(), animation_frame=0, binary_string=True)
# fig.show()

In [ ]:
X_V = []
X_S = []
with torch.no_grad():
    
    V, F = pc_r.get_fovs_VF(sweeps_fov)

    for v_, f_ in zip(V, F):
        x_s, _ = pc_r(f_.unsqueeze(0), v_.unsqueeze(0))
        threshold = np.percentile(x_s.detach().cpu().numpy(), 95)
        x_s = x_s.reshape(-1)
        v_ = v_.reshape(-1, 3)
        X_V.append(v_[x_s >= threshold])
        X_S.append(x_s[x_s >= threshold])

    # x_s, _ = pc_r(F, V)

    # threshold = np.percentile(x_s.detach().cpu().numpy(), 95)
    # print(threshold)
    # threshold = 0.85
    # x_s = x_s.reshape(-1)
    # V = V.reshape(-1, 3)
    # X_V = V[x_s >= threshold]

X_V = torch.cat(X_V, dim=0)
X_V = torch.unique(X_V, dim=0)

In [ ]:
x_v = X_V.cpu().numpy()
N = 50000
random_indices = torch.randperm(x_v.shape[0])[:N]

fig = go.Figure(data=[go.Scatter3d(x=x_v[random_indices,0], y=x_v[random_indices,1], z=x_v[random_indices,2], mode='markers', marker=dict(
        size=2,
        color=x_v[random_indices,2],                # set color to an array/list of desired values
        colorscale='jet',   # choose a colorscale
        opacity=1.0
    ))])
fig.show()